# <font color='blue'> Starting the environment  </font>
Once *QMWORKS*  has been installed the user should run the following command to initialize the environment:

To leave the environment the following command is used

# <font color='blue'> Packages and Settings </font> 
Currently **QMWORKS** offers an interface with the following simulation software:
* #### SCM (ADF and DTFB)
* #### CP2K
* #### ORCA
* #### GAMESS-US
* #### DIRAC

The inteaction between aforementioned packages and ``qmworks`` is carry out through a set of python function: 
*adf, dftb, cp2k, orca, gamess and dirac*.
These functions takes as arguments some ``settings`` describing the input and a molecular geometry as detailed below,

In [1]:
from qmworks import Settings
s = Settings()
s.basis = "DZP"
s.specific.adf.basis.core = "large"

As shown in the previous code snippet, the ``Settings`` ca be specified in two way: generic or specific. Generic keywords represent input properties that are present in most simulation packages like a *basis set* while *specific* keywords resemble the input structure of a given package.  

In [2]:
from qmworks import templates
inp = templates.geometry.overlay(s)

The *overlay* method takes as input a template containing a default set for different packages and also takes the arguments provided by the user, as shown schematically 
<img src="files/merged.png">

This overlay method merged the defaults for a given packages (*ADF* in this case) with the input supplied by the user, always given preference to the user input
<img src="files/result_merged.png" width="700">

**Templates** are stored inside the ``qmworks`` package as JSON files. Below it is shown the defaults for single point calculations

In [3]:
{
    "specific": {
        "adf": {
            "basis": {"type": "SZ"},
            "xc": {"lda": ""},
            "integration": {"accint": 4.0},
            "scf": {
            "converge": 1e-6,
            "iterations": 100} },
        "dftb": {
            "task": {"runtype": "SP"},
            "dftb": {"resourcesdir": "QUASINANO2015"} },
        "cp2k" : {
          "force_eval": {
              "dft": {
                  "basis_set_file_name": "",
                  "mgrid": {
                      "cutoff": 400,
                      "ngrids": 4
                  },
                  "potential_file_name": "",
                  "scf": {
                      "added_mos": "",
                      "eps_scf": 1e-06,
                      "max_scf": 200,
                      "scf_guess": "restart"
                  },
                  "xc": {
                      "xc_functional": "pbe"
                  }
              },
              "subsys": {
                  "cell": {
                      "periodic": "xyz"
                  },
                  "topology": {
                      "coordinate": "xyz",
                      "coord_file_name": ""}
                  }
              }
        },
        "dirac": {
            "DIRAC": "WAVEFUNCTION",
            "HAMILTONIAN": "LEVY-LEBLOND",
            "WAVE FUNCTION": "SCF"
        },
        "gamess": {
            "basis": {"gbasis": "sto", "ngauss": 3},
            "contrl": {"scftyp": "rhf", "dfttyp": "pbe"}
        },
        "orca": {
            "method": {
                "method": "dft",
                "functional": "lda"},
            "basis": {
                "basis": "sto_sz"}
        }
    }
}

{'specific': {'adf': {'basis': {'type': 'SZ'},
   'integration': {'accint': 4.0},
   'scf': {'converge': 1e-06, 'iterations': 100},
   'xc': {'lda': ''}},
  'cp2k': {'force_eval': {'dft': {'basis_set_file_name': '',
     'mgrid': {'cutoff': 400, 'ngrids': 4},
     'potential_file_name': '',
     'scf': {'added_mos': '',
      'eps_scf': 1e-06,
      'max_scf': 200,
      'scf_guess': 'restart'},
     'xc': {'xc_functional': 'pbe'}},
    'subsys': {'cell': {'periodic': 'xyz'},
     'topology': {'coord_file_name': '', 'coordinate': 'xyz'}}}},
  'dftb': {'dftb': {'resourcesdir': 'QUASINANO2015'},
   'task': {'runtype': 'SP'}},
  'dirac': {'DIRAC': 'WAVEFUNCTION',
   'HAMILTONIAN': 'LEVY-LEBLOND',
   'WAVE FUNCTION': 'SCF'},
  'gamess': {'basis': {'gbasis': 'sto', 'ngauss': 3},
   'contrl': {'dfttyp': 'pbe', 'scftyp': 'rhf'}},
  'orca': {'basis': {'basis': 'sto_sz'},
   'method': {'functional': 'lda', 'method': 'dft'}}}}

In [4]:
from plams import Molecule
acetonitrile = Molecule("files/acetonitrile.xyz")
print(acetonitrile)

  Atoms: 
    1         C      2.419290      0.606560      0.000000 
    2         C      1.671470      1.829570      0.000000 
    3         N      1.065290      2.809960      0.000000 
    4         H      2.000000      0.000000      1.000000 
    5         H      2.000000      0.000000     -1.000000 
    6         H      3.600000      0.800000      0.000000 



In [5]:
from qmworks import adf
job = adf(inp, acetonitrile)

### Extracting Properties
We can easily extract the *optimized geometry* from the *ADF* calculation using the following notation

In [6]:
optmized_mol_adf = job.molecule

We can use this geometry for further calculations using for instance another package

In [7]:
from qmworks import orca
s2 = Settings()
s2.specific.orca.main = "freq"
s2.specific.orca.basis.basis = 'sto_sz'
s2.specific.orca.method.functional = 'lda'
s2.specific.orca.method.method = 'dft'

job_freq = orca(s2, optmized_mol_adf)

frequencies = job_freq.frequencies

# <font color='blue'> Running the script </font>


## A little discussion about graphs
*qmworks* is meant to be used for both workflow generation and execution,

<img src="files/simple_graph.png">

In [8]:
from qmworks import run


### <font color='green'> result = run(frequencies) </font>

Once you run the script, as you we will see in the next section, an input file for the *ADF* job is created

Running in **Cartesius** or **Bazis** through the *Slurm* resource manager can be done using and script like

The Slurm output looks like: